In [1]:
import json
import numpy as np

In [3]:
REL_PATH_TO_NN_DATA = "../nn/data.json"

In [4]:
with open(REL_PATH_TO_NN_DATA, 'r') as f:
    data = json.load(f)

In [5]:
weights_list = np.array(data.get("weights")).T
weights_list.shape

(10, 785)

In [6]:
inputs_list = np.array(data.get("inputs"))
inputs_list.shape

(10000, 784)

In [7]:
format(123, "b").zfill(8)

'01111011'

In [8]:
np.binary_repr(123, width=8)

'01111011'

In [9]:
def convert_to_storage_format(data: list, data_bit_length: int, storage_bit_length: int) -> np.array:
    numbers_per_address = storage_bit_length / data_bit_length
    
    if not numbers_per_address.is_integer():
        raise ValueError("Storage bit length not divisble by data bit length!")
    
    storage_data = []
    bit_string = ""
    for index, number in enumerate(data):
        bit_string += format(number, "b").zfill(data_bit_length)
        
        if (index+1) % numbers_per_address == 0:
            storage_data.append(bit_string) # add the bit-string to the storage data
            bit_string = "" # reset the bitstring
            continue
        
        if len(data) == (index + 1) and len(bit_string) != storage_bit_length:
            bit_string =bit_string.ljust(storage_bit_length, "0") # pad with zeroes to the right
            storage_data.append(bit_string) # add the bit-string to the storage data
            break
            
    return np.array(storage_data)

In [10]:
# append a one at the start to include the bias multiplication (the inputs and weights are already reversed)
test_input = np.append([1], inputs_list[42])
test_input.shape

(785,)

In [11]:
input_storage_data = convert_to_storage_format(test_input, data_bit_length=3, storage_bit_length=12)
input_storage_data.shape

(197,)

In [12]:
weight_storage_data = []
for weights in weights_list:
    weight_storage_data.append(convert_to_storage_format(weights, data_bit_length=8, storage_bit_length=32))
    
weight_storage_data = np.ravel(np.array(weight_storage_data))
weight_storage_data.shape

(1970,)

### create the adresses for the weights

In [13]:
COUNT_OUTPUT_NEURONS = 10
COUNT_INPUT_MEMORY_ADRESSES = input_storage_data.shape[0]

In [21]:
NEURON_MEM_BIT_LENGTH = 4
INPUT_ADR_BIT_LENGTH = 8

weight_adresses = []

for neuron in range(0, COUNT_OUTPUT_NEURONS):
    for mem_adr in range(0, COUNT_INPUT_MEMORY_ADRESSES):
        neuron_bits = format(neuron, "b").zfill(NEURON_MEM_BIT_LENGTH)
        input_adr_bits = format(mem_adr, "b").zfill(INPUT_ADR_BIT_LENGTH)
        weight_adresses.append(neuron_bits + input_adr_bits)

weight_adresses = np.array(weight_adresses[::-1]) # reverse the adresses
weight_adresses

array(['100111000100', '100111000011', '100111000010', ...,
       '000000000010', '000000000001', '000000000000'], dtype='<U12')

In [43]:
def generate_vdhl_weight_array_allocation(arr_name: str, adr: np.array, data: np.array):
    with open("./weights_array_generation.txt", mode="w+") as file:

        # check the array format
        if not adr.shape[0] == data.shape[0]:
            raise ValueError("Shape of weight adresses and the weights itself must be the same!")

        for i in range(0, adr.shape[0]):
            print(f"{adr[i]} => {int(adr[i], 2)}")
            output_str = f'{int(adr[i], 2)} => "{data[i]}",\n'
            file.write(output_str)

### save the results

In [44]:
generate_vdhl_weight_array_allocation(
    arr_name="rom_arr",
    adr=weight_adresses,
    data=weight_storage_data
)

100111000100 => 2500
100111000011 => 2499
100111000010 => 2498
100111000001 => 2497
100111000000 => 2496
100110111111 => 2495
100110111110 => 2494
100110111101 => 2493
100110111100 => 2492
100110111011 => 2491
100110111010 => 2490
100110111001 => 2489
100110111000 => 2488
100110110111 => 2487
100110110110 => 2486
100110110101 => 2485
100110110100 => 2484
100110110011 => 2483
100110110010 => 2482
100110110001 => 2481
100110110000 => 2480
100110101111 => 2479
100110101110 => 2478
100110101101 => 2477
100110101100 => 2476
100110101011 => 2475
100110101010 => 2474
100110101001 => 2473
100110101000 => 2472
100110100111 => 2471
100110100110 => 2470
100110100101 => 2469
100110100100 => 2468
100110100011 => 2467
100110100010 => 2466
100110100001 => 2465
100110100000 => 2464
100110011111 => 2463
100110011110 => 2462
100110011101 => 2461
100110011100 => 2460
100110011011 => 2459
100110011010 => 2458
100110011001 => 2457
100110011000 => 2456
100110010111 => 2455
100110010110 => 2454
100110010101 

In [12]:
np.savetxt("weights.txt", weight_storage_data, delimiter="\n", fmt="%s")

In [13]:
np.savetxt("input.txt", input_storage_data, fmt="%s")